In [1]:
import py_entitymatching as em
import pandas as pd
import numpy as np

In [518]:
path_A = os.path.join('..','dataset','datasets','tracks.csv')
path_B = os.path.join('..','dataset','datasets','songs.csv')

# Read the CSV files
A = em.read_csv_metadata(path_A,low_memory=False) # setting the parameter low_memory to False  to speed up loading.
B = em.read_csv_metadata(path_B,low_memory=False)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [519]:
#Down sample the tables
sample_A, sample_B = em.down_sample(A, B, size=10000, y_param=1, show_progress=False)

# Set 'ID as the keys to the input tables
em.set_key(sample_A,'id')
em.set_key(sample_B,'id')

sample_A = sample_A.apply(lambda x: x.astype(str).str.lower())
sample_B = sample_B.apply(lambda x: x.astype(str).str.lower())

#Print lengths of the sampled tables
print(len(sample_A))
print(len(sample_B))

8592
10000


In [492]:
#def f(x):
#    return x['song_title'] + ' ' + x['artists']

In [520]:
#sample_A['combined'] = sample_A.apply(f, axis=1)
#sample_B['combined'] = sample_B.apply(f, axis=1)

sample_A.to_csv('tracks_sample.csv', index = False, sep = ',')
sample_B.to_csv('songs_sample.csv', index = False, sep = ',')

#Get headers of sampled tables
headers_A = list(A.columns)
headers_B = list(B.columns)

In [521]:
block_f = em.get_features_for_blocking(sample_A, sample_B)
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x129f10950>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x129f10ae8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_jar,id,id,None,None,jaro,<function id_id_jar at 0x129f10d08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_jwn,id,id,None,None,jaro_winkler,<function id_id_jwn at 0x129f106a8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x129f109d8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,id_id_jac_qgm_3_qgm_3,id,id,qgm_3,qgm_3,jaccard,<function id_id_jac_qgm_3_qgm_3 at 0x129f101e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,year_year_lev_dist,year,year,None,None,lev_dist,<function year_year_lev_dist at 0x129f10158>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,year_year_lev_sim,year,year,None,None,lev_sim,<function year_year_lev_sim at 0x129f10c80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,year_year_jar,year,year,None,None,jaro,<function year_year_jar at 0x129f10d90>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,year_year_jwn,year,year,None,None,jaro_winkler,<function year_year_jwn at 0x129f108c8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [522]:
path_A = 'tracks_sample.csv'
path_B = 'songs_sample.csv'

# Read the CSV files
sample_A = em.read_csv_metadata(path_A,key='id',low_memory=False)
sample_B = em.read_csv_metadata(path_B,key='id',low_memory=False)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [545]:
def title_function(x, y):

    x_title = str(x['song_title'])
    y_title = str(y['song_title'])
    
    if (x_title in y_title) or (y_title in x_title):
        return False
    else:
        x_split = x_title.split()
        y_split = y_title.split()
        
        intersection = len(set(x_split) & set(y_split))
        union = len(set(x_split) | set(y_split))
        
        #if(union - intersection > 3):
        #    return True
        #else:
        if(intersection / union < 0.25):
            return True
        else:
            return False

In [546]:
def artists_function(x, y):

    x_artists = str(x['artists'])
    y_artists = str(y['artists'])
    
    if (x_artists in y_artists) or (y_artists in x_artists):
        return False
    else:
        x_split = x_artists.split()
        y_split = y_artists.split()
        
        intersection = len(set(x_split) & set(y_split))
        union = len(set(x_split) | set(y_split))
        
        #if(union - intersection > 4):
        #    return True
        #else:
        if(intersection / union < 0.25):
            return True
        else:
            return False

In [547]:
rb = em.RuleBasedBlocker()
ob = em.OverlapBlocker()
bb = em.BlackBoxBlocker()

# remove pairs that don't share similar titles
rule1 = ['song_title_song_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1']
#rule2 = ['artists_artists_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.2']
rb.add_rule(rule1, block_f)
#rb.add_rule(rule2, block_f)

C1 = rb.block_tables(sample_A, sample_B, l_output_attrs=['song_title','year','artists'], r_output_attrs=['song_title','year','artists'], show_progress=False)

bb.set_black_box_function(title_function)
C2 = bb.block_candset(C1)

bb.set_black_box_function(artists_function)
C3 = bb.block_candset(C2)

0%                          100%
[##############################] | ETA: 00:01:42 | ETA: 00:01:37 | ETA: 00:01:34 | ETA: 00:01:33 | ETA: 00:01:30 | ETA: 00:01:29 | ETA: 00:01:27 | ETA: 00:01:24 | ETA: 00:01:21 | ETA: 00:01:17 | ETA: 00:01:13 | ETA: 00:01:08 | ETA: 00:01:05 | ETA: 00:01:01 | ETA: 00:00:57 | ETA: 00:00:53 | ETA: 00:00:50 | ETA: 00:00:46 | ETA: 00:00:42 | ETA: 00:00:38 | ETA: 00:00:34 | ETA: 00:00:30 | ETA: 00:00:27 | ETA: 00:00:23 | ETA: 00:00:19 | ETA: 00:00:15 | ETA: 00:00:11 | ETA: 00:00:07 | ETA: 00:00:03 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:01:55
0%  100%
[#] | ETA: 00:00:00 | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:01:19 | ETA: 00:01:00 | ETA: 00:00:51 | ETA: 00:00:47 | ETA: 00:00:43 | ETA: 00:00:40 | ETA: 00:00:37 | ETA: 00:00:35 | ETA: 00:00:33 | ETA: 00:00:31 | ETA: 00:00:29 | ETA: 00:00:27 | ETA: 00:00:25 | ETA: 00:00:24 | ETA: 00:00:22 | ETA: 00:00:20 | ETA: 00:00:19 | ETA: 00:00:17 | ETA: 00:00:16 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:45
0%                          100%
[##############################] | ETA: 00:00:21 | ETA: 00:00:18 | ETA: 00:00:15 | ETA: 00:00:13 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00

In [548]:
print(len(C3))
C3.head(50)

1119


,_id,ltable_id,rtable_id,ltable_song_title,ltable_year,ltable_artists,rtable_song_title,rtable_year,rtable_artists
340,340,457485,241086,cooler than me,2013.0,mike posner+big sean / eric richard holljes / mike posner,cooler than me,2010,mike posner
558,558,454747,944347,lend your love to me tonight,1982.0,greg lake+pete sinfield+emerson lake+palmer,lend your love to me tonight,1977,greg lake
3075,3075,492493,351793,one bass hit,1946.0,ray brown+gil fuller+dizzy gillespie+his orchestra,one bass hit,1957,dizzy gillespie
3139,3139,729591,61089,angel,1945.0,harry warren+lucille bremer ) to herself,guardian angel,0,m
3144,3144,483864,61089,sweet little angel,1998.0,big mama thornton,guardian angel,0,m
3153,3153,708024,61089,angel face,1965.0,maurizio graf,guardian angel,0,m
3897,3897,493017,558871,starwood in aspen,2013.0,john denver,sneaking in,0,john swihart
4983,4983,694431,500408,move up to the country,2009.0,dutch mason,resort to the beat,0,dutch master
5955,5955,490675,399923,sexy bitch,2010.0,david guetta )+david guetta+jean-claude sindres+sandy vee,sexy bitch (featuring akon; abel ramos atlanta with love mix remix; explicit),0,david guetta - akon
6056,6056,632937,741085,rockin all over the world,2003.0,john fogerty+status quo,all over,0,john holt


In [549]:
# debug the blocker
dbg = em.debug_blocker(C2, sample_A, sample_B, output_size=50)

In [550]:
dbg

,_id,similarity,ltable_id,rtable_id,ltable_song_title,ltable_artists,rtable_song_title,rtable_artists
0,0,0.714286,420488,475597,sick of it all,the stickmen,the reason,sick of it all
1,1,0.625000,420488,821949,sick of it all,the stickmen,the bland within,sick of it all
2,2,0.611111,649222,805895,up up down down left right left right b a b a start,john orr+kid overdrive,by surprise,up up down down left right left right b a start
3,3,0.600000,669174,35275,toko,momo wandel soumah,signoya,momo wandel soumah
4,4,0.600000,356065,884993,yarabi,bembeya jazz national,wouloukoro,bembeya jazz national
5,5,0.600000,303114,392103,tsunami,manic street preachers,yourself,manic street preachers
6,6,0.600000,618127,887608,lackluster,poi dog pondering,sticky,poi dog pondering
7,7,0.600000,641609,212901,toro,el columpio asesino,moscas,el columpio asesino
8,8,0.600000,657790,518628,america,the mercury project,compromise,the mercury project
9,9,0.600000,691477,102790,the turning point,NaN,before the dawn,turning point


In [551]:
#Sample the result set C6
S= em.sample_table(C3,400)

In [552]:
# label the sampled data

# CRITERIA:
#
# - same song name (excluding version information)
#    - same or common artist(s) -> MATCH
#    - completely different artist(s) -> MISMATCH
#    - artist(s) missing -> MISMATCH

G = em.label_table(S, label_column_name='gold_labels')

Column name (gold_labels) is not present in dataframe


In [554]:
G.to_csv('labeled_data.csv', index = False, sep = ',')